In [3]:
!pip install torch torchvision tqdm
import torch, os
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

    100% |████████████████████████████████| 519.5MB 31kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5968e000 @  0x7fb3ed1f31c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 1.7MB/s 
    100% |████████████████████████████████| 51kB 11.7MB/s 
    100% |████████████████████████████████| 2.0MB 2.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [4]:
transform = transforms.Compose([transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [0]:
def train(epoch, trainloader, optimizer, criterion):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader), 0):
        # get the inputs
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('epoch %d training loss: %.3f' %
            (epoch + 1, running_loss / (len(trainloader))))

In [0]:
def test(testloader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
                                    100 * correct / total))

In [0]:
def classwise_test(testloader, model):
########################################################################
# class-wise accuracy

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))


In [13]:
num_epochs = 10
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=1)#32-1+1=32
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#16
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=1)#16
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)#8
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=28, kernel_size=1)#8
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#4
        self.fc1 = nn.Linear(in_features=28*4*4, out_features=480)
        self.fc2 = nn.Linear(in_features=480, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 28*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 2.140


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 10 training loss: 1.012


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 57 %


100%|██████████| 2500/2500 [00:14<00:00, 167.42it/s]

Accuracy of plane : 48 %
Accuracy of   car : 62 %
Accuracy of  bird : 49 %
Accuracy of   cat : 36 %
Accuracy of  deer : 49 %
Accuracy of   dog : 42 %
Accuracy of  frog : 67 %
Accuracy of horse : 69 %
Accuracy of  ship : 74 %
Accuracy of truck : 69 %
Finished Training


In [14]:
num_epochs = 5
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)#32-5+1=28
        self.drop1 = nn.Dropout(p=0.2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=4, stride=2)#13
        self.drop2 = nn.Dropout(0.2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=96, kernel_size=4)#10
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.fc1 = nn.Linear(in_features=96*5*5, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=96)
        self.fc3 = nn.Linear(in_features=96, out_features=10)


    def forward(self, x):
        x = self.drop1(F.relu(self.conv1(x)))
        x = self.drop2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 96*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 1.839


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 5 training loss: 1.025


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 59 %


100%|██████████| 2500/2500 [00:15<00:00, 158.58it/s]

Accuracy of plane : 71 %
Accuracy of   car : 59 %
Accuracy of  bird : 42 %
Accuracy of   cat : 35 %
Accuracy of  deer : 50 %
Accuracy of   dog : 60 %
Accuracy of  frog : 73 %
Accuracy of horse : 63 %
Accuracy of  ship : 69 %
Accuracy of truck : 69 %
Finished Training


In [15]:
num_epochs = 1
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)#32-5+1=28
        self.drop1 = nn.Dropout(p=0.2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=4, stride=2)#13
        self.drop2 = nn.Dropout(0.2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=96, kernel_size=4)#10
        self.drop3 = nn.Dropout(p=0.2)
        self.conv4 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=1)#10
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.fc1 = nn.Linear(in_features=256*5*5, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=96)
        self.fc3 = nn.Linear(in_features=96, out_features=10)


    def forward(self, x):
        x = self.drop1(F.relu(self.conv1(x)))
        x = self.drop2(F.relu(self.conv2(x)))
        x = self.drop3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = x.view(-1, 256*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 2.073


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 34 %


100%|██████████| 2500/2500 [00:18<00:00, 138.52it/s]

Accuracy of plane : 45 %
Accuracy of   car : 37 %
Accuracy of  bird : 23 %
Accuracy of   cat : 12 %
Accuracy of  deer : 39 %
Accuracy of   dog : 43 %
Accuracy of  frog : 27 %
Accuracy of horse : 26 %
Accuracy of  ship : 28 %
Accuracy of truck : 63 %
Finished Training


In [16]:
num_epochs = 1
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)#32-5+1=28
        self.drop1 = nn.Dropout(p=0.2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=4, stride=2)#13
        self.drop2 = nn.Dropout(0.2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=96, kernel_size=4)#10
        self.conv4 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=1)#10
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.fc1 = nn.Linear(in_features=256*5*5, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=96)
        self.fc3 = nn.Linear(in_features=96, out_features=10)


    def forward(self, x):
        x = self.drop1(F.relu(self.conv1(x)))
        x = self.drop2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool4(F.relu(self.conv4(x)))
        x = x.view(-1, 256*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 2.007


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 39 %


100%|██████████| 2500/2500 [00:19<00:00, 125.08it/s]


Accuracy of plane : 36 %
Accuracy of   car : 46 %
Accuracy of  bird : 30 %
Accuracy of   cat :  4 %
Accuracy of  deer : 40 %
Accuracy of   dog : 47 %
Accuracy of  frog : 37 %
Accuracy of horse : 57 %
Accuracy of  ship : 60 %
Accuracy of truck : 35 %
Finished Training


In [17]:
num_epochs = 20
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)#32-5+1=28
        self.drop1 = nn.Dropout(p=0.2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=4, stride=2)#13
        self.drop2 = nn.Dropout(0.2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=96, kernel_size=4)#10
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.fc1 = nn.Linear(in_features=96*5*5, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=96)
        self.fc3 = nn.Linear(in_features=96, out_features=10)


    def forward(self, x):
        x = self.drop1(F.relu(self.conv1(x)))
        x = self.drop2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 96*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 1.871


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 20 training loss: 0.200


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 60 %


100%|██████████| 2500/2500 [00:21<00:00, 116.28it/s]

Accuracy of plane : 66 %
Accuracy of   car : 71 %
Accuracy of  bird : 51 %
Accuracy of   cat : 51 %
Accuracy of  deer : 57 %
Accuracy of   dog : 50 %
Accuracy of  frog : 64 %
Accuracy of horse : 51 %
Accuracy of  ship : 74 %
Accuracy of truck : 68 %
Finished Training


In [0]:
num_epochs = 20
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)#32-5+1=28
        self.drop1 = nn.Dropout(p=0.2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=4, stride=2)#13
        self.drop2 = nn.Dropout(0.2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=96, kernel_size=4)#10
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.fc1 = nn.Linear(in_features=96*5*5, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=96)
        self.fc3 = nn.Linear(in_features=96, out_features=10)


    def forward(self, x):
        x = self.drop1(F.relu(self.conv1(x)))
        x = self.drop2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 96*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

In [18]:
num_epochs = 5
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.drop = nn.Dropout(p=0.2)
        
        self.fc1 = nn.Linear(in_features=3*32*32, out_features=2048)
        self.fc2 = nn.Linear(in_features=2048, out_features=1024)
        self.fc3 = nn.Linear(in_features=1024, out_features=512)
        self.fc4 = nn.Linear(in_features=512, out_features=84)
        self.fc5 = nn.Linear(in_features=84, out_features=10)


    def forward(self, x):
        x = x.view(-1, 3*32*32)
        x = self.drop(F.relu(self.fc1(x)))
        x = self.drop(F.relu(self.fc2(x)))
        x = self.drop(F.relu(self.fc3(x)))
        x = self.drop(F.relu(self.fc4(x)))
        x = self.fc5(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 1.986


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 5 training loss: 1.603


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 42 %


100%|██████████| 2500/2500 [00:18<00:00, 133.77it/s]


Accuracy of plane : 31 %
Accuracy of   car : 75 %
Accuracy of  bird : 24 %
Accuracy of   cat : 37 %
Accuracy of  deer : 22 %
Accuracy of   dog : 27 %
Accuracy of  frog : 61 %
Accuracy of horse : 35 %
Accuracy of  ship : 65 %
Accuracy of truck : 43 %
Finished Training


In [22]:
num_epochs = 10
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=5)#32-5+1=28
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#14
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=5)#14-5+1=10
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3)#5-3+1=3
        #self.conv4 = nn.Conv2d(in_channels=192, out_channels=192, kernel_size=1)#1
        #self.conv5 = nn.Conv2d(in_channels=192, out_channels=10, kernel_size=1)#1
        self.fc1 = nn.Linear(in_features=192*3*3, out_features=192)
        self.fc2 = nn.Linear(in_features=192, out_features=10)
        #self.fc3 = nn.Linear(in_features=96, out_features=10)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        
        x = x.view(-1, 192*3*3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 1.712


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 10 training loss: 0.277


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 69 %


100%|██████████| 2500/2500 [00:20<00:00, 124.24it/s]

Accuracy of plane : 73 %
Accuracy of   car : 89 %
Accuracy of  bird : 55 %
Accuracy of   cat : 31 %
Accuracy of  deer : 59 %
Accuracy of   dog : 63 %
Accuracy of  frog : 83 %
Accuracy of horse : 72 %
Accuracy of  ship : 84 %
Accuracy of truck : 83 %
Finished Training


In [0]:
num_epochs = 10
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=5)#32-5+1=28
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#14
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=5)#14-5+1=10
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3)#5-3+1=3
        self.fc1 = nn.Linear(in_features=192*3*3, out_features=800)
        self.fc2 = nn.Linear(in_features=800, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        
        x = x.view(-1, 192*3*3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')